This notebook includes examples of how to use some core operations defined in our paper.

## Warmup

In [1]:
from pprint import pprint

database/concert_singer.sqlite contains an sqlite3 database in the original Spider Dataset release. We call this database $o$ and print its basic properties

In [2]:
from sql_util.dbinfo import get_all_db_info_path
original_database_path = 'database/concert_singer.sqlite'
# now we extract information from an sqlite database path
table_column2properties, column_references, table_column2elements = get_all_db_info_path(original_database_path)

table_column2elements is a dictionary mapping from a table's cololumn name to a list of elements in that column.
We print all columns in the "stadium" table

In [3]:
for k, v in table_column2elements.items():
    if k[0] == 'stadium':
        print(k, v)

('stadium', 'STADIUM_ID') [1, 2, 3, 4, 5, 6, 7, 9, 10]
('stadium', 'LOCATION') ['Raith Rovers', 'Ayr United', 'East Fife', "Queen's Park", 'Stirling Albion', 'Arbroath', 'Alloa Athletic', 'Peterhead', 'Brechin City']
('stadium', 'NAME') ["Stark's Park", 'Somerset Park', 'Bayview Stadium', 'Hampden Park', 'Forthbank Stadium', 'Gayfield Park', 'Recreation Park', 'Balmoor', 'Glebe Park']
('stadium', 'CAPACITY') [10104, 11998, 2000, 52500, 3808, 4125, 3100, 4000, 3960]
('stadium', 'HIGHEST') [4812, 2363, 1980, 1763, 1125, 921, 1057, 837, 780]
('stadium', 'LOWEST') [1294, 1057, 533, 466, 404, 411, 331, 400, 315]
('stadium', 'AVERAGE') [2106, 1477, 864, 730, 642, 638, 637, 615, 552]


Suppose we have a gold_query $g$ that is associated with the schema of original_database_path $o$

In [4]:
gold_query = 'select location from stadium where capacity < 100000 ;'

## Section 2: Calculating Denotation Equivalence

Now we look at the denotation 􏰑􏰑$[[g]]_{o}$ of gold_query $g$ on the original_database_path (database $o$). 

In [5]:
from sql_util.run import exec_db_path
gold_flag, gold_denotation = exec_db_path(original_database_path, gold_query)
print('gold denotation')
print(gold_denotation)

gold denotation
[('Raith Rovers',), ('Ayr United',), ('East Fife',), ("Queen's Park",), ('Stirling Albion',), ('Arbroath',), ('Alloa Athletic',), ('Peterhead',), ('Brechin City',)]


A false prediction $q$ = 'select location from stadium ;' will have the same denotation on $o$; hence $o$ fails to distinguish $g$ and $q$. function result_eq compares whether two denotation is equivalent

In [6]:
q = 'select location from stadium ;'
q_flag, q_denotation = exec_db_path(original_database_path, q)
from sql_util.eq import result_eq
print("same" if result_eq(q_denotation, gold_denotation, order_matters=False) else "different", "denotation")

same denotation


## Section 3.1: Neighbor Queries $N_{g}$

This following function generates the set of neighbor queries $N_{g}$ of the gold_query of $g$.
The function takes in the database ($o$) path with schema associated with the gold query $g$ and the gold query $g$ as argument

In [7]:
from fuzz.neighbor import generate_neighbor_queries_path
neighbors = generate_neighbor_queries_path(original_database_path, gold_query)
import random
random.shuffle(neighbors)
pprint(neighbors[:10])
print('There are', len(neighbors), 'neighbor queries generated for the gold query.')

['select AVERAGE from stadium where capacity < 100000 ;',
 'select location from stadium where capacity < -73632 ;',
 'select location from stadium where NAME < 100000 ;',
 'select location from stadium where100000 ;',
 'select location from stadium where 100000 ;',
 'select location from stadium ;',
 'select location from stadium where AVERAGE < 100000 ;',
 'select location from stadium;',
 'select CAPACITY from stadium where capacity < 100000 ;',
 'select location from stadium where capacity < 99999 ;']
There are 40 neighbor queries generated for the gold query.


## Section 4.1: Sampling Databases from $\mathcal{I}_{g}$

The below function randomly generates an sqlite3 database $w$ (a database sample from $\mathcal{I}_{g}$, defined in Section 4) at "sampled_database_w_path" based on the constant used in the gold_query $g$ and the database ($o$) schema associated with the gold_query (original_database_path).

In [8]:
from fuzz.fuzz import generate_random_db_with_queries_wrapper
sampled_database_w_path = 'database/example_generation.sqlite'
generate_random_db_with_queries_wrapper((original_database_path, sampled_database_w_path, [gold_query], {}))

now fuzzing based on database database/concert_singer.sqlite, target path database/example_generation.sqlite.


We now look at the capacity column of the stadium table, which should contain close constant variants of 100000, which is used in the gold_query $g$.

In [9]:
table_column2properties, column_references, table_column2elements = get_all_db_info_path(sampled_database_w_path)
print(table_column2elements[('stadium', 'CAPACITY')][:5])

[2036384813, 10104, 10103, 3959, 3961]


Now we look at the denotation 􏰑􏰑$[[g]]_{w}$ of gold_query $g$ on the sampled_database_w_path (database $w$)

In [10]:
from sql_util.run import exec_db_path
gold_flag, gold_denotation = exec_db_path(sampled_database_w_path, gold_query)
print('gold denotation')
print(gold_denotation)

gold denotation
[('cz4LX0bQbZkH5iEPS0',), ('H691(Fx7bp',), ('Raith Rovers',), ('Raith Rovers',), ('B6k7twptl5I06ySqhO',), ('fyjgGGi',), ('Brechin CityOYYJ)',), ('Brechin City',), ('VaXFMBrechin City',), ('Arbroath',), ('ofvjQW8EPtGDDQ4IhV',), ('Brechin City',), ('Brechin City',), ('hMaArbroathIt4',), ('Raith Rovers',), ("Queen's Park 5S08",), ('Brechin Cit',), ('Brechin CityYw 7)',), ('foZ (GC0P',), ('Ou(AsL7wXZz',), ('Brechin City',)]


$w$ should be able to distinguish $g$ and $q$.

In [11]:
q_flag, q_denotation = exec_db_path(sampled_database_w_path, q)
print("same " if result_eq(q_denotation, gold_denotation, order_matters=False) else "different", "denotation")

different denotation


## Section 4.2 Constructing $S_{g}$

Now we greedily construct a testsuite $S_{g}$ that can distinguish all the neighbor queries. This algorhtm can also be seen in the Appendix A.1

In [12]:
# the set of neighbors that have not been distinguished
undistinguished_neighbors = set(neighbors)
testsuite = set()
T = 1000
for t in range(T):
    # stop the algorithm when all the neighbors have been distinguished
    if len(undistinguished_neighbors) == 0:
        break
    # randomly sample a database based in the gold query and the database sechema associated with the query
    sampled_database_w_path = 'database/db%d.sqlite' % t
    generate_random_db_with_queries_wrapper((original_database_path, sampled_database_w_path, [gold_query], {}))
    
    # check whether the sampled database can distinguish the gold from the neighbor
    for u in set(undistinguished_neighbors):
        gold_flag, gold_denotation = exec_db_path(sampled_database_w_path, gold_query)
        u_flag, u_denotation = exec_db_path(sampled_database_w_path, u)
        # add the database path to the test suite if it can distinguish a neighbor from the gold
        if not result_eq(gold_denotation, u_denotation, order_matters=False):
            undistinguished_neighbors.remove(u)
            testsuite.add(sampled_database_w_path)

# Note: the gold query is not very complicated, so it does not need a lot of samples to distinguish all neighbors
print('databases in the test suite')
print(testsuite)

now fuzzing based on database database/concert_singer.sqlite, target path database/db0.sqlite.
now fuzzing based on database database/concert_singer.sqlite, target path database/db1.sqlite.
databases in the test suite
{'database/db1.sqlite', 'database/db0.sqlite'}
